## 고급 구조 패턴
- 배치 정규화
- 깊이별 분리 합성곱

## 배치 정규화
- 정규화(normalization)
    - 입력 샘플들을 균일하게 만드는 광범위한 방법
    - normalized_data = (data - np.mean(data, axis=...)) / np.std(data, axis=...)
    - 이전 까지는 모델에 입력하기 전단계에서 정규화했음
        - 하지만 데이터 정규화는 네트워크에서 일어나는 모든 변환 후에도 고려되어야!
- **BatchNormalization**
    - https://arxiv.org/abs/1502.03167
    - 트레이닝 하는 동안 평균과 분산이 바뀌더라도 적응해서 데이터 정규화
    - 트레이닝 과정에 사용된 배치 데이터의 평균과 분산에 대한 **지수 이동 평균**을 내부에 유지
        - 즉, 입력 배치(들)의 평균과 표준 편차를 Exponential Moving Average를 통해 계산함으로써 전체 데이터셋에 대한 평균/표준편차를 대신한다.
            - 이값은 테스트 데이터에 정규화가 적용될 때도 사용됨
            - v = v * momentum + v_new * (1 - momentum)으로 계산되며 mementum 기본값은 0.99
        - 깊은 네트워크 구성 가능
            - 입력에 비해 활성화 함수의 출력이 너무 작거나 큰 경우 
            - 예를들어 변화율이 급격히 작아진다면 역전파되는 그래디언트도 줄어들게 되므로,
            - 배치 정규화를 쓰면 입출력의 분포를 유지하도록 도와주어 그래디언트 전파가 잘된다
        - 그냥 간단히 **`model.add(layers.BatchNormalization())`** 만 추가하면 됨
            - 일반적으로 Conv 나 Dense 다음에 추가한다
            - axis=-1 default (channels_last 인 경우만)
            - axis=1 => channels_first인 경우

## 배치 정규화 이후
- 배치 재정규화(batch renormalization)
    - https://arxiv.org/abs/1702.03275

## 초기화 방법
- lecun_normal
    - initializers.lecun_normal() 입력 유닛 개수의 역수에 대한 제곱근을 표준 편차로 하는 절단 정규 분포로 가중치 초기화 하는 방법

## 깊이별 분리 합성곱
- Conv2D보다 더 가벼워(파라미터 적음), 더 빨라(부동 소수 연산이 더 적음), 모델의 성능을 높일 수도 있는 레이어
- depthwise separable convolution
    - SeparableConv2D
        1. 입력 채널별로 따로따로 공간 방향의 합성곱 수행 후
        2. 그 이후에 pointwise conv(1x1 conv)를 통해 출력 채널 합쳐준다
    - **공간 특성의 학습과 채널 방향 특성의 학습을 분리**하는 효과
    - 입력에서 공간상의 위치는 상관관계가 크지만 채널별로는 매우 독립적일 경우 타당
    - 작고 더 빠른 모델을 만들어줌
- 엑셉션(Xception)에 깊이별 분리 합성곱에 대한 이론적 배경 소개함

In [2]:
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3,
                                activation='relu',
                                input_shape=(height, width, channels)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Using TensorFlow backend.
